In [1]:
import keras
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop
from lsuv_init import LSUVinit

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import backend as K
import keras.backend.tensorflow_backend as KTF
K.clear_session()
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   
session = tf.Session(config=config)
KTF.set_session(session)

Using TensorFlow backend.


In [2]:
(x_train,y_train),(x_test,y_test) = cifar100.load_data()

In [3]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [4]:
x_train = x_train / 255
x_test = x_test / 255
y_train = keras.utils.np_utils.to_categorical(y_train, 100)
y_test = keras.utils.np_utils.to_categorical(y_test, 100)

In [5]:
model = Sequential()

In [6]:
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

In [7]:
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [8]:
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

In [9]:
model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 128)       0         
__________

In [11]:
# plot_model(model,to_file='model2.png')

In [12]:
# opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
# opt = keras.optimizers.Adam(lr=0.001, 
#                             beta_1=0.9, 
#                             beta_2=0.999, 
#                             epsilon=10e-8, 
#                             decay=0.0
# )
opt = keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              min_delta=0.0001, 
                              cooldown=0, 
                              min_lr=0
)

In [15]:
tensorboard = TensorBoard(log_dir='./logs_sgd',
                histogram_freq=0,
                batch_size=64,
                write_graph=True,
                write_grads=False,
                write_images=True,
                embeddings_freq=0, 
                embeddings_layer_names=None, 
                embeddings_metadata=None
)

In [16]:
model_checkpoing = ModelCheckpoint(filepath='./tmp_models/weights_sgd.hdf5', 
                                   monitor='val_loss', 
                                   verbose=0, 
                                   save_best_only=True, 
                                   save_weights_only=False, 
                                   mode='auto', 
                                   period=1
)

In [17]:
earlystop = EarlyStopping(monitor='val_loss', 
                          patience=20, 
                          verbose=1, 
                          mode='auto'
)

In [18]:
datagen = ImageDataGenerator(rotation_range=10, 
                             zoom_range=0.15, 
                             width_shift_range=0.2, 
                             height_shift_range=0.2, 
#                              zca_whitening=True,
#                              zca_epsilon=1e-06,
                             horizontal_flip=True
)

In [19]:
datagen.fit(x_train)

In [ ]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=64), 
                              steps_per_epoch=x_train.shape[0] // 64, 
                              epochs=300, 
                              validation_data=(x_test, y_test), 
                              workers=4, 
                              verbose=1, 
                              callbacks=[tensorboard, model_checkpoing, reduce_lr, earlystop]
)

Epoch 1/300
781/781 [==============================] - 59s 76ms/step - loss: 4.8829 - acc: 0.0160 - val_loss: 4.3437 - val_acc: 0.0581
Epoch 2/300
781/781 [==============================] - 57s 73ms/step - loss: 4.6253 - acc: 0.0270 - val_loss: 4.2242 - val_acc: 0.0810
Epoch 3/300
781/781 [==============================] - 57s 74ms/step - loss: 4.4810 - acc: 0.0342 - val_loss: 4.1814 - val_acc: 0.0891
Epoch 4/300
781/781 [==============================] - 59s 75ms/step - loss: 4.3960 - acc: 0.0413 - val_loss: 4.1192 - val_acc: 0.1060
Epoch 5/300
781/781 [==============================] - 60s 77ms/step - loss: 4.3262 - acc: 0.0471 - val_loss: 4.0708 - val_acc: 0.1136
Epoch 6/300
781/781 [==============================] - 60s 77ms/step - loss: 4.2696 - acc: 0.0514 - val_loss: 4.0286 - val_acc: 0.1168
Epoch 7/300
781/781 [==============================] - 62s 80ms/step - loss: 4.2248 - acc: 0.0581 - val_loss: 3.9894 - val_acc: 0.1256
Epoch 8/300
781/781 [==============================] - 

In [ ]:
model.save('full_model2_sgd.h5')

In [ ]:
fig = plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
#
fig.savefig('accuracy_sgd.png')

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
#
fig.savefig('loss_sgd.png')